In [20]:
import requests
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler
# Get the current date and time
current_time = datetime.utcnow()

# Calculate the start and end dates for the 7-day period
start_time = current_time - timedelta(days=7)
end_time = current_time - timedelta(days=1)
api_key = 'c2c135e363fa4eaa851105600232303'
# Convert the start and end dates to the required format
start_date = start_time.strftime("%Y-%m-%d")
end_date = end_time.strftime("%Y-%m-%d")

city = input('Enter a city/state name: ')
# Construct the API request URL
url = f"https://api.weatherapi.com/v1/history.json?key={api_key}&q={city}&dt={start_date}&end_dt={end_date}"
response = requests.get(url)

# Extract the temperature, humidity, and summary data from the response
data = response.json()

# Parse the response and extract the temperature, humidity, and summary data for each day
data = response.json()['forecast']['forecastday']
temp_data = [day['day']['avgtemp_c'] for day in data]
humidity_data = [day['day']['avghumidity'] for day in data]
summary_data = [day['day']['condition']['text'] for day in data]

encoder = LabelEncoder()
summary_data = encoder.fit_transform(summary_data)

# Create a DataFrame to store the data
""" forecasted_data = pd.DataFrame({'Temperature (C)': temp_data, 'Humidity': humidity_data, 'Summary': summary_data}) """
forecasted_data = pd.DataFrame({'Temperature (C)': temp_data, 'Humidity': humidity_data, 'Summary': summary_data}, columns=['Temperature (C)', 'Humidity', 'Summary'])

# Print the first few rows of the DataFrame
print(forecasted_data.head())
# Load the "weatherHistory.csv" file into a Pandas DataFrame
df = pd.read_csv('weatherHistory.csv')

# Convert the date column to the desired format
df['Date'] = df['Formatted Date'].str.slice(stop=10)



df.drop_duplicates(inplace=True)

df['Date'] = pd.to_datetime(df['Date'], utc=True)

df.dropna(inplace=True)

df = df[(df['Temperature (C)'] > -273.15) & (df['Temperature (C)'] < 60)]

df = df.dropna(subset=['Temperature (C)', 'Humidity', 'Summary'])
# Now the "Date" column should have the desired format
print(df.head())

encoder = LabelEncoder()
df['Summary'] = encoder.fit_transform(df['Summary'])
df['Temperature (C)'] = encoder.fit_transform(df['Temperature (C)'])
df['Humidity'] = encoder.fit_transform(df['Humidity'])




# Define the features and target variable
features = ['Temperature (C)', 'Humidity','Summary']
targets = ['Temperature (C)', 'Humidity','Summary']

# Split the data into training and testing sets
train = df[df['Date'] < '2016-01-01']
test = df[df['Date'] >= '2016-01-01']


X_train = train[features]
Y_train = train[targets]

X_test = test[features]
Y_test = test[targets]

# Preprocess your features
scaler = StandardScaler()
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
# Train a random forest regression model
model = RandomForestRegressor(n_estimators=100, random_state=42)  
model.fit(X_train, Y_train)

# Use the model to make predictions for the next 7 days
future_dates = pd.date_range(start=current_time, periods=7, freq='D')
future_weather = forecasted_data.values
future_weather_df = pd.DataFrame(future_weather, index=future_dates, columns=forecasted_data.columns)

predictions = model.predict(future_weather_df)

# Print the predicted weather for each day
for i, date in enumerate(future_dates):
    print(f'On {date.date()}, the weather is predicted to be: ')
    print(f"Temperature (C): {predictions[i][0]}")
    print(f"Humidity: {predictions[i][1]}")
    print(f"Summary: {predictions[i][2]}")



   Temperature (C)  Humidity  Summary
0             10.8      81.0        0
1             10.5      88.0        0
2             10.3      72.0        1
3             10.1      85.0        0
4             11.3      72.0        2
                  Formatted Date        Summary Precip Type  Temperature (C)  \
0  2006-04-01 00:00:00.000 +0200  Partly Cloudy        rain         9.472222   
1  2006-04-01 01:00:00.000 +0200  Partly Cloudy        rain         9.355556   
2  2006-04-01 02:00:00.000 +0200  Mostly Cloudy        rain         9.377778   
3  2006-04-01 03:00:00.000 +0200  Partly Cloudy        rain         8.288889   
4  2006-04-01 04:00:00.000 +0200  Mostly Cloudy        rain         8.755556   

   Apparent Temperature (C)  Humidity  Wind Speed (km/h)  \
0                  7.388889      0.89            14.1197   
1                  7.227778      0.86            14.2646   
2                  9.377778      0.89             3.9284   
3                  5.944444      0.83            14

e:\Program Files\Python311\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
